In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torch import optim
import sys
import random
import math
import time
from tqdm import tqdm
from sklearn.metrics import precision_recall_fscore_support, f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

from transformers import BertTokenizer, AutoTokenizer
from transformers import BertModel, AutoModel, AutoModelForSequenceClassification
from transformers import AdamW, get_linear_schedule_with_warmup

from torch.utils.tensorboard import SummaryWriter

use_cuda = True if torch.cuda.is_available() else False
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# use_cuda=False
# device='cpu'

torch.autograd.set_detect_anomaly(True)
torch.backends.cudnn.benchmark = True
np.random.seed(0)
torch.manual_seed(0)

base_model = 'twitter-xlm-roberta-base-sentiment'
model_list = ['bert-base-uncased', 'bert-base-multilingual-uncased', 'google/muril-base-cased', 'xlm-roberta-base',
              'ai4bharat/indic-bert','cardiffnlp/twitter-xlm-roberta-base','cardiffnlp/twitter-xlm-roberta-base-sentiment',
              'cardiffnlp/twitter-roberta-base', 'cardiffnlp/twitter-roberta-base-sentiment',
              'cardiffnlp/twitter-roberta-base-hate', 'roberta-base']
model_path = '/mnt/saved_models/'
results_path = '/mnt/saved_results/'

In [3]:
lang = 'portuguese'
model_choice = 6

In [6]:
writer = SummaryWriter(log_dir="/home/jupyter/tboard/" + base_model + "_" + lang)
device

device(type='cuda')

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_list[model_choice])

MAX_SEQ_LEN = 128

label_idx = 1
text_idx = 0

class HateData(Dataset):
    def __init__(self, data_path, split='train', lang='bengali', aug_prob=0.2, flip_prob=0.5):
        self.split = split
        # self.data = pd.read_parquet(data_path + lang + "_" + split + ".parquet", engine='fastparquet')
        # self.data = pd.read_csv(data_path, sep=',')
        # self.data = pd.read_csv(data_path + lang + "_" + split + ".tsv", sep='\t')
        self.data = pd.read_csv(data_path + split + "_" + lang + ".tsv", sep='\t', lineterminator='\n')
        
        if self.split == 'train':
            self.label2data = {0:[], 1:[], 2:[]}
            # self.data = self.data[self.data['language'] == lang]
            
            for i in tqdm(range(len(self.data))):
                row = self.data.iloc[i]
                self.label2data[row[label_idx]].append(row[text_idx])
            self.aug_prob = aug_prob
            self.flip_prob = flip_prob

    def __len__(self):
        return len(self.data)

    
    def __getitem__(self, index):
        if torch.is_tensor(index):
            index = index.tolist()

        data = self.data.iloc[index]

        labels = data[label_idx]
        text = data[text_idx]
        inputs = tokenizer(text, padding='max_length', truncation=True, max_length=MAX_SEQ_LEN)
        # print(inputs)
        input_ids = inputs['input_ids']
        token_type_ids = np.zeros(MAX_SEQ_LEN)#inputs['token_type_ids']#
        attn_mask = inputs['attention_mask']
        
        aug_text = text  
        labels_aug = labels
        
        if self.split == 'train' and labels == 1:
            if np.random.uniform() < self.aug_prob:
                aug_text = np.random.choice(self.label2data[0])
         
                if np.random.uniform() < self.flip_prob:
                    aug_text = aug_text + " [SEP] " + text
                else:
                    aug_text = text + " [SEP] " + aug_text 
            labels_aug = 1
      
        inputs_aug = tokenizer(aug_text, padding='max_length', truncation=True, max_length=MAX_SEQ_LEN)
        # print(inputs)
        input_ids_aug = inputs_aug['input_ids']
        token_type_ids_aug = np.zeros(MAX_SEQ_LEN)#inputs_aug['token_type_ids']#
        attn_mask_aug = inputs_aug['attention_mask']

        input_ids = torch.tensor(np.vstack([input_ids, input_ids_aug]), dtype=torch.long).view(2, MAX_SEQ_LEN)
        token_type_ids = torch.tensor(np.vstack([token_type_ids, token_type_ids_aug]), dtype=torch.long).view(2, MAX_SEQ_LEN)
        attn_mask = torch.tensor(np.vstack([attn_mask, attn_mask_aug]), dtype=torch.long).view(2, MAX_SEQ_LEN)
        labels = torch.tensor(np.vstack([labels, labels_aug]), dtype=torch.long).view(2)


        return input_ids, attn_mask, token_type_ids, labels


# train_data = HateData(data_path="/home/jupyter/data/implicit-hate-corpus/", lang='latent')
# dataload = DataLoader(train_data, batch_size=4)

# for i in (dataload):
#     print(i[0].shape)
#     print(i[1].shape)
#     print(i[2].shape)
#     print(i[3].shape)
#     break




In [8]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()

        H1, H2, num_class = 768, 128, 2
        self.bert = AutoModel.from_pretrained(model_list[model_choice])
        
        # for param in self.bert.parameters():
        #     param.requires_grad = False

        self.clf = nn.Sequential(
            nn.Linear(H1, H2),
            nn.ReLU(),
            nn.Linear(H2, H2),
            nn.ReLU(),
            nn.Linear(H2, num_class)
        )

        
    def forward(self, input_ids, attn_mask, token_type_ids):  
        outputs = self.bert(input_ids, attn_mask)#, token_type_ids)
        cls_emb = outputs.pooler_output # (batch, 768)
        logits = self.clf(cls_emb) # (batch, num_class)
        return logits


In [9]:
loss_fn = nn.CrossEntropyLoss()#

In [10]:
def train(input_ids, attn_mask, token_type_ids, label, model, model_opt, scdl):

    model_opt.zero_grad()

    batch_size = input_ids.shape[0]
    seq_len = input_ids.shape[1]

    loss = 0.0

    if use_cuda:
        input_ids = input_ids.to(device)
        attn_mask = attn_mask.to(device)
        token_type_ids = token_type_ids.to(device)
        label = label.to(device)

    # label = label.flatten()
    
    logits = model(input_ids[:,0,:], attn_mask[:,0,:], token_type_ids[:,0,:])
    logits_aug = model(input_ids[:,1,:], attn_mask[:,1,:], token_type_ids[:,1,:])

    loss = loss_fn(logits, label[:,0]) + loss_fn(logits_aug, label[:,1])

    # if torch.isnan(loss):
    #     pass
    # else:
    loss.backward()
    # torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) # clip gradients to prevent exploding
    model_opt.step()
    scdl.step()
    # print(loss)
    return float(loss.item())



In [11]:
def evaluate(input_ids, attn_mask, token_type_ids, label, model, mode='train'):
   
    batch_size = input_ids.shape[0]
    seq_len = input_ids.shape[1]


    with torch.no_grad():
        if use_cuda:
            input_ids = input_ids.to(device)
            attn_mask = attn_mask.to(device)
            token_type_ids = token_type_ids.to(device)
            label = label.to(device)

        # label = label.flatten()
        
        logits = model(input_ids[:,0,:], attn_mask[:,0,:], token_type_ids[:,0,:])
        loss = loss_fn(logits, label[:,0])
        
        if mode == 'train':
            return float(loss.item())
        
        preds = torch.argmax(logits, dim=1).flatten()
        # acc = (preds == label).cpu().numpy().mean() * 100

        return float(loss.item()), preds.cpu().numpy()
        



In [12]:
df_test = pd.read_csv("/home/jupyter/multilingual/test_data/test_portuguese.tsv", sep='\t', lineterminator='\n')
gt_labels = np.array(df_test['label'])

In [13]:
len(gt_labels)

284

In [14]:
def trainIters(model, epochs, train_loader, test_loader, learning_rate=3e-5, log_step=168, valid_step=168, mode='train'):

    model_opt = AdamW(model.parameters(), lr=learning_rate, eps=1e-8)
    num_train_steps = (len(train_loader)*epochs) 
    scdl = get_linear_schedule_with_warmup(model_opt, num_warmup_steps=int(0.1*num_train_steps), num_training_steps=num_train_steps)

    print("Initialised optimizer and lr scheduler")

    # valid_best_loss = [] 
    best_acc = 0.0 
    tot = len(train_data) // train_loader.batch_size
    tot_val = len(val_data) // test_loader.batch_size
    plot_steps = 0
    
    for epoch in range(epochs):
        train_loss_total = 0.0
        train_step = 0
        # Training
        
        model.train()        
        for entry in tqdm(train_loader, total=tot, position=0, leave=True):
            loss = train(entry[0], entry[1], entry[2], entry[3], model, model_opt, scdl)
            plot_steps += 1
            train_step += 1
            # if not math.isnan(loss) :      
            train_loss_total = train_loss_total + loss
            
            train_loss = train_loss_total / train_step
            
            if plot_steps % log_step == 0:
                writer.add_scalar("Train Loss", train_loss, plot_steps)
            
            if (plot_steps % valid_step == 0) or (plot_steps >= num_train_steps - 1):
                model.eval()
                test_pred = []

                for entry in tqdm(test_loader, total=tot_val, position=0, leave=True):
                    loss_v, pred_v = evaluate(entry[0], entry[1], entry[2], entry[3], model, mode='test')
                    # if not math.isnan(loss) :      
                    test_pred.extend([pd for pd in pred_v])

                # val_acc = (test_pred == gt_labels).mean().item()
                val_acc = f1_score(gt_labels, test_pred, average='macro')
                print("Validation F1: " + str(val_acc))
                writer.add_scalar("Val F1", val_acc, plot_steps)


                #   Save best model
                # state = {
                #         'epoch': epoch,
                #         'state_dict': model.state_dict(),
                #         'optimizer': model_opt.state_dict(),
                #         'loss': train_loss,
                #         'scheduler': scdl.state_dict(),
                # }


                if val_acc > best_acc:
                    torch.save(model.state_dict(), model_path + "model_" + base_model + "_" + lang + "_easymix_mono_redo" + ".pth") 
                    print("Model saved for step: " + str(plot_steps))
                    best_acc = val_acc         

                model.train()
            writer.flush()
                

        print('epoch: '+str(epoch))
        print('total loss: '+str(train_loss_total/tot))

        # wr_train = open(results_path + "train_loss_" + base_model + ".txt", "a")
        # wr_train.write("epoch " + str(epoch) + ": " + str(train_loss_total/tot) + "\n")
        # wr_train.close()


        

In [15]:
train_data = HateData(data_path="/home/jupyter/multilingual/train_data/", split='train', lang=lang)
val_data = HateData(data_path="/home/jupyter/multilingual/test_data/", split='test', lang=lang)

100%|██████████| 5386/5386 [00:00<00:00, 12640.44it/s]


In [16]:
BS = 16
# weights = [1.0]*15383
# weights.extend([0.5]*(len(train_data) - 15383))
# sampler = WeightedRandomSampler(weights, num_samples=20000)

dataload = DataLoader(train_data, batch_size=BS, shuffle=True)
dataload_val = DataLoader(val_data, batch_size=BS, shuffle=False)

In [17]:
(len(train_data)/16)//2

168.0

In [18]:
# model = Classifier()
# model = model.float()
# # model = nn.DataParallel(model)#, device_ids = [2, 3]
# model = model.to(device)
model = Classifier()
# model = nn.DataParallel(model)
model.load_state_dict(torch.load("/mnt/saved_models/model_twitter-xlm-roberta-base-sentiment_multilingual_redo.pth", map_location=device))
model = model.to(device)

Some weights of the model checkpoint at cardiffnlp/twitter-xlm-roberta-base-sentiment were not used when initializing XLMRobertaModel: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.out_proj.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaModel were not initialized from the model checkpoint at cardiffnlp/twitter-xlm-roberta-base-sentiment and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to 

In [19]:
trainIters(model, 5, dataload, dataload_val)

Initialised optimizer and lr scheduler


18it [00:00, 20.97it/s]                        s]


Validation F1: 0.718214032600992


 50%|█████     | 168/336 [02:19<11:46,  4.21s/it]

Model saved for step: 168


...

Validation F1: 0.6734511027819672


337it [04:16,  1.32it/s]                         


epoch: 2
total loss: 0.32165907951448824


 15%|█▌        | 52/336 [00:39<03:34,  1.33it/s]


KeyboardInterrupt: 

In [ ]:
device

######################## TESTING ######################

In [12]:
model = Classifier()
# model = nn.DataParallel(model)
model.load_state_dict(torch.load("/mnt/saved_models/model_twitter-roberta-base-sentiment_" + lang + "_easymix" + "_redo.pth", map_location=device))
model = model.to(device)

Some weights of the model checkpoint at cardiffnlp/twitter-xlm-roberta-base-sentiment were not used when initializing XLMRobertaModel: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaModel were not initialized from the model checkpoint at cardiffnlp/twitter-xlm-roberta-base-sentiment and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to 

In [13]:
lang #= 'hindi'

'spanish'

In [15]:
# test_data = HateData(data_path="/home/jupyter/data/test_data/bq_test_" + lang + "_process_10k.csv")
test_data = HateData(data_path="/home/jupyter/multilingual/test_data/", split='test', lang=lang)
test_loader = DataLoader(test_data, batch_size=1, shuffle=False)

In [16]:
model.eval()
test_loss = []
test_pred = []

# wr = open(results_path + "test_prediction_" + base_model + "_" + lang + "_process_10k.txt", "w")     
wr = open(results_path + "test_prediction_" + base_model + "_" + lang + ".txt", "w")    
for entry in tqdm(test_loader, total=len(test_data)//test_loader.batch_size, position=0, leave=True):
    v_loss, v_pred = evaluate(entry[0], entry[1], entry[2], entry[3], model, mode='test')
    test_loss.append(v_loss)
    test_pred.append(v_pred)
    wr.write(str(v_pred)+"\n")

test_loss = np.mean(test_loss)#.item()

print("Test Loss: ", test_loss)

wr.close()

100%|██████████| 1600/1600 [00:35<00:00, 45.37it/s]

Test Loss:  0.4558552019682247


In [17]:
# df_test = pd.read_csv("/home/jupyter/data/test_data/bq_test_" + lang + "_process_10k.csv", sep=',')
df_test = pd.read_csv("/home/jupyter/multilingual/test_data/test_spanish.tsv", sep='\t', lineterminator='\n')
# df_test = pd.read_csv("/home/jupyter/data/test_data/hx_test.tsv", sep='\t')
# gt_labels = np.array(df_test['class'])
# mp = {'hate':1, 'normal':0}
# df_test['label'].replace(mp, inplace=True)
gt_labels = np.array(df_test['label'])

In [18]:
print(classification_report(gt_labels, test_pred, digits=4))

              precision    recall  f1-score   support

           0     0.8328    0.8372    0.8350       940
           1     0.7664    0.7606    0.7635       660

    accuracy                         0.8056      1600
   macro avg     0.7996    0.7989    0.7993      1600
weighted avg     0.8054    0.8056    0.8055      1600



In [ ]:
ConfusionMatrixDisplay.from_predictions(gt_labels, np.array(test_pred))

In [ ]:
latent2 - 80.14, 78.44

In [19]:
test_pred # 79.43,76.89

[array([0]),
 array([2]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([2]),
 array([2]),
 array([2]),
 array([1]),
 array([0]),
 array([0]),
 array([1]),
 array([2]),
 array([2]),
 array([1]),
 array([1]),
 array([0]),
 array([0]),
 array([2]),
 array([2]),
 array([2]),
 array([0]),
 array([2]),
 array([2]),
 array([2]),
 array([2]),
 array([0]),
 array([2]),
 array([2]),
 array([0]),
 array([2]),
 array([0]),
 array([2]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([1]),
 array([1]),
 array([0]),
 array([0]),
 array([0]),
 array([2]),
 array([0]),
 array([2]),
 array([2]),
 array([0]),
 array([0]),
 array([1]),
 array([2]),
 array([2]),
 array([1]),
 array([0]),
 array([2]),
 array([2]),
 array([0]),
 array([2]),
 array([2]),
 array([2]),
 array([1]),
 array([1]),
 array([0]),
 array([2]),
 array([2]),
 array([2]),
 array([0]),
 array([0]),
 array([1]),
 array([0]),
 array([2]),
 array([2]),
 array([1]),
 array([2]),
 array([1]),
 array([0]),
 array([2]),

In [ ]:
78.52, 76.13 - rob-tws + easymix

In [ ]:
78.84, 76.32 - rob-twh